# 21 PR.AVATECT

- GAOZの組みを変えている  
  -> 多数決方式は不可能

- Random Forestによる識別率が0.36と、あまり出ていないので一応試し程度

### 担当者と日付
2024/10/2 渡邉健

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]
# fmt: on


### C_0 ~ C_9までの読み込み

In [3]:
from os import listdir

ATTACK_TARGET = "21"

c_data_list = []

files = listdir("../data")

for i in range(10):
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,F,18,12,136,0,0,0,0,0,0,4,0,2,0,0,0,0
1,F,25,17,967,0,4,3,2,0,0,4,4,1,0,1,0,0
2,F,18,6,445,0,0,0,4,1,0,2,0,0,2,3,4,0
3,M,50,5,523,0,0,0,5,2,1,2,2,3,0,0,0,5
4,M,1,17,321,3,4,4,3,4,1,0,3,3,5,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,25,4,752,2,1,0,5,0,2,4,0,0,1,3,0,0
9996,M,18,13,532,3,2,0,5,2,4,0,0,1,0,1,5,3
9997,M,35,6,210,2,1,0,4,0,2,5,5,2,2,0,0,3
9998,M,1,12,494,4,2,4,5,1,3,0,1,1,3,3,5,3


### GAOZ+MovieIDとRatingの組み合わせを取得

In [4]:
data_frames = []
for c_data_df in c_data_list:
    df_long = c_data_df.melt(id_vars=['Gender', 'Age', 'Occupation', 'ZIP-code'],
                      var_name='MovieID', value_name='Rating')
    data_frames.append(df_long)
    # print(df_long)

data = pd.concat(data_frames, ignore_index=True)
data = data.dropna(subset=['Rating'])

data

,Gender,Age,Occupation,ZIP-code,MovieID,Rating
0,F,18,12,136,260,0
1,F,25,17,967,260,0
2,F,18,6,445,260,0
3,M,50,5,523,260,0
4,M,1,17,321,260,3
...,...,...,...,...,...,...
1099995,F,25,4,462,2193,4
1099996,F,18,13,108,2193,4
1099997,F,35,6,722,2193,3
1099998,F,1,12,977,2193,0


### 特徴量と目的変数の設定

In [5]:
# Ratingを整数型に変換
data['Rating'] = data['Rating'].astype(int)

# ステップ2: 特徴量と目的変数の設定
X = data[['Gender', 'Age', 'Occupation', 'ZIP-code', 'MovieID']]
y = data['Rating']

X

,Gender,Age,Occupation,ZIP-code,MovieID
0,F,18,12,136,260
1,F,25,17,967,260
2,F,18,6,445,260
3,M,50,5,523,260
4,M,1,17,321,260
...,...,...,...,...,...
1099995,F,25,4,462,2193
1099996,F,18,13,108,2193
1099997,F,35,6,722,2193
1099998,F,1,12,977,2193


In [6]:
categorical_features = ['Gender', 'Age', 'Occupation', 'ZIP-code', 'MovieID']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = encoder.fit_transform(X[categorical_features])

encoded_feature_names = encoder.get_feature_names_out(categorical_features)
X_prepared = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# デバッグ情報: エンコーディング後の特徴量の確認
print("エンコーディング後の特徴量の数:", X_prepared.shape[1])
print("エンコーディング後の特徴量名の一部:", encoded_feature_names[:])

エンコーディング後の特徴量の数: 571
エンコーディング後の特徴量名の一部: ['Gender_F' 'Gender_M' 'Age_1' 'Age_18' 'Age_25' 'Age_35' 'Age_45'
 'Age_50' 'Age_56' 'Occupation_0' 'Occupation_1' 'Occupation_2'
 'Occupation_3' 'Occupation_4' 'Occupation_5' 'Occupation_6'
 'Occupation_7' 'Occupation_8' 'Occupation_9' 'Occupation_10'
 'Occupation_11' 'Occupation_12' 'Occupation_13' 'Occupation_14'
 'Occupation_15' 'Occupation_16' 'Occupation_17' 'Occupation_18'
 'Occupation_19' 'Occupation_20' 'ZIP-code_6' 'ZIP-code_10' 'ZIP-code_12'
 'ZIP-code_13' 'ZIP-code_15' 'ZIP-code_16' 'ZIP-code_17' 'ZIP-code_18'
 'ZIP-code_19' 'ZIP-code_20' 'ZIP-code_21' 'ZIP-code_22' 'ZIP-code_23'
 'ZIP-code_24' 'ZIP-code_26' 'ZIP-code_27' 'ZIP-code_28' 'ZIP-code_29'
 'ZIP-code_30' 'ZIP-code_31' 'ZIP-code_33' 'ZIP-code_37' 'ZIP-code_38'
 'ZIP-code_40' 'ZIP-code_41' 'ZIP-code_42' 'ZIP-code_45' 'ZIP-code_48'
 'ZIP-code_49' 'ZIP-code_50' 'ZIP-code_53' 'ZIP-code_54' 'ZIP-code_56'
 'ZIP-code_60' 'ZIP-code_61' 'ZIP-code_62' 'ZIP-code_63' 'ZIP-code_64'
 'ZIP

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size=0.05, random_state=42)

### 学習

In [8]:
# ステップ5: モデルの構築と学習（分類モデル）
model = RandomForestClassifier(n_estimators=100, random_state=42, verbose=2)
model.fit(X_train, y_train)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  7.3min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

RandomForestClassifier(random_state=42, verbose=2)

In [9]:
import pickle

pickle.dump(model, open(f"../models/{ATTACK_TARGET}_random_forest.pkl", 'wb'))

In [10]:
# ステップ6: モデルの評価
y_pred = model.predict(X_test)
print("分類レポート:")
print(classification_report(y_test, y_pred))

# ステップ7: 各クラスの予測確率の取得
y_pred_proba = model.predict_proba(X_test)

# 予測確率の例を表示
prob_df = pd.DataFrame(y_pred_proba, columns=model.classes_)
print("最初の10件の予測確率:")
print(prob_df.head(10))

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    3.0s


分類レポート:
              precision    recall  f1-score   support

           0       0.44      0.73      0.55     19434
           1       0.26      0.16      0.20      6682
           2       0.26      0.16      0.20      6724
           3       0.26      0.16      0.20      7589
           4       0.25      0.19      0.22      7967
           5       0.26      0.16      0.19      6604

    accuracy                           0.36     55000
   macro avg       0.29      0.26      0.26     55000
weighted avg       0.32      0.36      0.32     55000



[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.9s


最初の10件の予測確率:
          0         1         2         3         4         5
0  0.755000  0.020000  0.090000  0.050000  0.020000  0.065000
1  0.100000  0.058333  0.086333  0.313333  0.223333  0.218667
2  0.025000  0.300000  0.440000  0.190000  0.010000  0.035000
3  0.365000  0.060000  0.015000  0.110000  0.190000  0.260000
4  0.696333  0.159000  0.030000  0.000000  0.050000  0.064667
5  0.000000  0.010000  0.090000  0.000000  0.890000  0.010000
6  0.160000  0.030000  0.020000  0.620000  0.010000  0.160000
7  0.692500  0.005000  0.000000  0.030000  0.052500  0.220000
8  0.621000  0.094000  0.010000  0.015000  0.020000  0.240000
9  0.500000  0.270000  0.030000  0.103333  0.040000  0.056667


In [11]:
# ステップ8: 新しいGAOZを使った予測
new_data = pd.DataFrame({
    'Gender': ['F'],
    'Age': [45],
    'Occupation': [12],
    'ZIP-code': [253],
    'MovieID': ["1525"]
})

# カテゴリ変数のエンコーディング
new_encoded_features = encoder.transform(new_data[categorical_features])
new_X = pd.DataFrame(new_encoded_features, columns=encoded_feature_names)

# 予測と予測確率の取得
new_y_pred = model.predict(new_X)
new_y_pred_proba = model.predict_proba(new_X)

print(f"予測されたRating: {new_y_pred[0]}")
print("各評価値に対する予測確率:")
for rating, prob in zip(model.classes_, new_y_pred_proba[0]):
    print(f"Rating {rating}: {prob * 100:.2f}%")

予測されたRating: 2
各評価値に対する予測確率:
Rating 0: 13.00%
Rating 1: 2.00%
Rating 2: 43.17%
Rating 3: 1.00%
Rating 4: 40.83%
Rating 5: 0.00%


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s


## 推測攻撃

In [12]:
MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']

Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Yank Schiell,M,50,16,104,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Prince Tibbotts,M,1,4,132,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Martainn Mellem,M,50,12,971,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dorey Pittson,M,50,0,954,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dav Wakeling,M,50,10,146,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
def fill_nan_with_prediction(row):
    for movie_id in MOVIE_IDS:
        if pd.isna(row[movie_id]):
            # 予測用のデータを準備
            new_data = pd.DataFrame({
                'Gender': [row['Gender']],
                'Age': [int(row['Age'])],
                'Occupation': [int(row['Occupation'])],
                'ZIP-code': [int(row['ZIP-code'])],
                'MovieID': [movie_id]
            })

            # カテゴリ変数のエンコーディング
            new_encoded_features = encoder.transform(new_data[categorical_features])
            new_X = pd.DataFrame(new_encoded_features, columns=encoded_feature_names)

            # 予測と予測確率の取得
            new_y_pred = model.predict(new_X)
            new_y_pred_proba = model.predict_proba(new_X)

            # 最大の予測確率を取得
            max_prob = np.max(new_y_pred_proba)

            # 予測確率が62%以上の場合のみ値を埋める
            if max_prob >= 0.55:
                # print(f"MovieID {movie_id}の予測確率: {max_prob * 100:.2f}%")
                row[movie_id] = new_y_pred[0]
            # それ以外の場合はnanのままにする

    return row

# NaNを条件付きで埋める
Ba = Ba.apply(fill_nan_with_prediction, axis=1)

# 結果の確認
# print(Ba.head())
# print(Ba.isnull().sum().sum())

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Do

In [32]:
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3439,3440,3466,3479,3489,3877,3889,Closest_Bb_Index,Hamming_Distance,Non_NaN_Col_Total
1,Yank Schiell,M,50,16,104,NaN,NaN,NaN,1.0,0.0,...,NaN,0.0,0.0,0.0,NaN,NaN,0.0,1,4,16
2,Prince Tibbotts,M,1,4,132,0.0,1.0,4.0,0.0,NaN,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,1,21,35
3,Martainn Mellem,M,50,12,971,NaN,4.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,24,8,22
4,Dorey Pittson,M,50,0,954,NaN,4.0,NaN,4.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,1,11,26
5,Dav Wakeling,M,50,10,146,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,7,3,13


In [33]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,0,0,3,1,2,5,3,0,0,*,...,1,0,4,3,0,0,0,2,0,4
1,5,3,4,4,0,0,0,0,5,1,...,0,0,0,0,0,0,4,2,0,0
2,0,1,5,2,0,3,0,0,0,1,...,0,3,2,0,0,0,4,1,0,1
3,4,3,4,1,0,2,0,3,*,4,...,1,4,5,1,2,4,5,0,0,4
4,0,4,5,2,5,5,0,4,0,2,...,0,0,2,0,0,0,3,3,0,1
5,1,2,1,3,2,0,1,4,0,3,...,3,5,2,4,4,4,0,4,4,3
6,3,0,0,5,1,*,1,1,5,2,...,4,2,3,0,0,2,0,3,0,4
7,3,1,5,2,4,0,0,0,0,5,...,0,0,1,0,0,0,3,3,0,0
8,3,3,3,2,3,1,0,1,0,3,...,0,0,2,4,1,0,1,0,2,0
9,3,4,0,0,4,4,2,0,1,0,...,4,3,3,0,0,3,2,3,4,5


In [34]:

def nan_aware_hamming_distance(a, b):
    # 両方のSeriesで共通するインデックスのみを使用
    common_index = a.index.intersection(b.index)
    a = a[common_index]
    b = b[common_index]
    # NaNでない値のみを比較
    valid_indices = ~(a.isna() | b.isna())
    return np.sum(a[valid_indices] != b[valid_indices])

def find_closest_row_nan_aware(Ba_row):
    distances = []
    for idx, Bb_row in Bb.iterrows():
        distance = nan_aware_hamming_distance(Ba_row, Bb_row)
        distances.append((idx, distance))
    
    min_index, min_distance = min(distances, key=lambda x: x[1])
    return pd.Series({'Closest_Bb_Index': min_index, 'Hamming_Distance': min_distance})

# 結果を計算
results = Ba.apply(find_closest_row_nan_aware, axis=1)

# 結果をBaに追加
Ba['Closest_Bb_Index'] = results['Closest_Bb_Index']
Ba['Hamming_Distance'] = results['Hamming_Distance']

# Non_NaN_Col_Totalカラムを追加
Ba['Non_NaN_Col_Total'] = Ba.count(axis=1)

In [35]:
Ba

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3439,3440,3466,3479,3489,3877,3889,Closest_Bb_Index,Hamming_Distance,Non_NaN_Col_Total
1,Yank Schiell,M,50,16,104,NaN,NaN,NaN,1.0,0.0,...,NaN,0.0,0.0,0.0,NaN,NaN,0.0,17,10,25
2,Prince Tibbotts,M,1,4,132,0.0,1.0,4.0,0.0,NaN,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,1,24,40
3,Martainn Mellem,M,50,12,971,NaN,4.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,0.0,NaN,24,12,28
4,Dorey Pittson,M,50,0,954,NaN,4.0,NaN,4.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,1,15,30
5,Dav Wakeling,M,50,10,146,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,7,7,18
6,Morgan Jamot,M,50,12,750,NaN,4.0,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,0.0,0.0,17,9,25
7,Blisse Tottem,F,1,10,919,NaN,1.0,4.0,NaN,NaN,...,0.0,0.0,3.0,NaN,NaN,0.0,NaN,34,18,35
8,Lynnell Brody,F,25,18,26,NaN,NaN,4.0,NaN,NaN,...,0.0,0.0,NaN,0.0,NaN,0.0,NaN,1,9,25
9,Brew Estoile,M,50,14,331,NaN,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,2,8,21
10,Niki Murden,F,45,1,23,NaN,1.0,NaN,2.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1,8,24


In [36]:
row_index = 2

gender = Ba.iloc[row_index]["Gender"]
age = Ba.iloc[row_index]["Age"]
occupation = Ba.iloc[row_index]["Occupation"]
zipcode = Ba.iloc[row_index]["ZIP-code"]

print(gender, age, occupation, zipcode)

filtered_C_list = []

for c in c_data_list:
    filtered_C_0 = c[(c['Gender'] == gender) & 
                      (c['Age'] == int(age)) & 
                      (c['Occupation'] == int(occupation)) & 
                      (c['ZIP-code'] == int(zipcode))]
    filtered_C_list.append(filtered_C_0)

len(filtered_C_list)

M 50 12 971


10

In [37]:
merged_rows = []

for i, filtered_df in enumerate(filtered_C_list):
    if not filtered_df.empty:
        # 各フィルタリングされたデータフレームから1行だけ取得
        row = filtered_df.iloc[0]
        
        # ALL_HEADERに基づいて新しい行を作成
        new_row = pd.Series(index=ALL_HEADER)
        
        # データを転送
        for col in ALL_HEADER:
            if col in filtered_df.columns:
                new_row[col] = row[col]
            else:
                new_row[col] = np.nan
        
        # 新しい行をリストに追加
        merged_rows.append(new_row)

# リストからデータフレームを作成
merged_df = pd.DataFrame(merged_rows, columns=ALL_HEADER)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889


In [38]:
def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
random_values = {}
for column in merged_df.columns:
    random_values[column] = select_random_value(merged_df[column])

# 結果を新しいデータフレームとして作成
result_df = pd.DataFrame([random_values])
result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
closest_indices, distances = find_closest_hamming_index(result_df, Bb)
closest_indices, distances

NameError: name 'find_closest_hamming_index' is not defined